In [1]:
import os

# đọc video từ thư mục data/video
video_path = "data/video"
video_list = os.listdir(video_path)

In [2]:
# thêm vào danh sách video các video lật ngược name+f
new_video_list = video_list.copy()

for video in video_list:
    new_video_list.append(video[:-4] + "f.mp4")

video_list = new_video_list

In [4]:
import cv2
import sys

sys.path.append("../module/")
from init import init_csv
from pose_landmarker import extract_pose_features

list = []
for video_name in video_list:
    # Nếu tên video có chữ f ở cuối thì xóa chữ f đi và lật ngược video
    if video_name[-5] == "f":
        # Đọc video
        cap = cv2.VideoCapture(os.path.join(video_path, video_name[:-5] + ".mp4"))
    else:
        # Đọc video
        cap = cv2.VideoCapture(os.path.join(video_path, video_name))

    # Kiểm tra xem video có được mở thành công không
    if not cap.isOpened():
        print("Không thể mở video.")
        exit()

    # Khởi tạo csv_writer
    csv_writer = init_csv(f"data/csv/{video_name.split('.')[0]}.csv")

    # Đọc từng frame từ video
    while True:
        ret, frame = cap.read()

        # Kiểm tra nếu có chữ f ở cuối tên video thì lật ngược frame
        if video_name[-5] == "f":
            frame = cv2.flip(frame, 1)

        # Kiểm tra xem có còn frame nào để đọc không
        if not ret:
            break

        # Trích xuất đặc trưng từ frame
        landmarks = extract_pose_features(frame)

        if landmarks is not None:
            # Chuyển landmarks thành mảng 1 chiều
            list = [j for i in landmarks for j in i]
            # Ghi dữ liệu xuống file csv
            csv_writer.writerow(list)

        # Thời gian chờ giữa các frame (để đảm bảo đọc video với tốc độ 30 fps)
        wait_time = int(1000 / 30)

# Đóng cửa sổ hiển thị và giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()

In [7]:
import pandas as pd
import csv

# Khởi tạo csv_writer
csv_file = open("data/data.csv", "w", newline="")  # Tạo file csv
csv_writer = csv.writer(csv_file)  # Khởi tạo csv_writer

# Khởi tạo header cho file csv
header = [f"{i}" for i in range(1, 337)] + ["label"]
csv_writer.writerow(header)

for video_name in video_list:
    # Đọc file csv
    csv_reader = pd.read_csv(f"data/csv/{video_name.split('.')[0]}.csv")

    # Lấy 28 dòng đầu tiên của file csv và chuyển nó thành mảng 1 chiều
    list = [j for i in csv_reader.head(28).values for j in i]

    # Nếu tên video có số 0 đầu thì thêm số 0 và cuối list
    if video_name.split(".")[0][0] == "0":
        list.append(0)
    else:
        list.append(1)

    # Ghi dữ liệu xuống file csv
    csv_writer.writerow(list)

# Đóng file csv
csv_file.close()

In [8]:
# Đọc file data.csv
csv_reader = pd.read_csv("data/data.csv")

# In ra 5 dòng đầu tiên của file csv
csv_reader

,1,2,3,4,5,6,7,8,9,10,...,328,329,330,331,332,333,334,335,336,label
0,0.482556,0.301600,-0.016666,0.478341,0.489260,-0.000020,0.519976,0.788471,0.122996,0.430674,...,0.474062,0.483298,0.000002,0.516464,0.785509,0.101732,0.431066,0.781687,0.098530,0.0
1,0.429633,0.518555,-0.188891,0.454323,0.495486,0.000037,0.594170,0.760206,0.029811,0.335250,...,0.439054,0.317143,-0.000015,0.663881,0.771235,-0.008332,0.289918,0.694929,0.317332,0.0
2,0.479542,0.452572,-0.579912,0.522247,0.320358,0.000055,0.621352,0.834840,0.080860,0.365188,...,0.566404,0.537541,0.000080,0.603351,0.875083,0.057695,0.530981,0.749985,0.424799,0.0
3,0.502769,0.598824,-0.312952,0.562964,0.701259,0.000136,0.597893,0.916841,-0.091828,0.481447,...,0.629994,0.806218,-0.000025,0.570793,0.894468,0.088400,0.567647,0.868667,0.339632,0.0
4,0.414003,0.573399,-0.001384,0.597498,0.699475,-0.000021,0.566169,0.817170,0.004108,0.565621,...,0.596602,0.631569,0.000035,0.562060,0.805773,-0.040946,0.561076,0.788998,0.512586,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.564337,0.516398,-0.091470,0.441500,0.608379,0.000027,0.275301,0.812072,0.015818,0.372683,...,0.439468,0.604488,0.000035,0.264256,0.816169,0.002736,0.263929,0.704241,0.111803,0.0
158,0.569864,0.499144,-0.109137,0.396515,0.586703,-0.000034,0.155435,0.737928,0.417455,0.165271,...,0.313770,0.369554,0.000107,0.105447,0.758616,0.207106,0.076398,0.794593,0.007621,0.0
159,0.333527,0.457074,-0.276835,0.227306,0.390680,-0.000018,0.169473,0.456785,0.668721,0.173566,...,0.576807,0.654229,-0.000012,0.410618,0.584765,0.283575,0.426473,0.554281,0.375113,1.0
160,0.433554,0.414548,-0.111500,0.430231,0.517660,0.000005,0.420926,0.649469,0.140842,0.436376,...,0.529367,0.632668,0.000023,0.471256,0.680181,0.336675,0.449191,0.664091,0.476416,1.0
